In [3]:
import ray
ray.shutdown()

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_backfill as pb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl
import bar_meta as bm
import filters as ft
# import market_cluster as mc

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]


import ray
ray.init(ignore_reinit_error=True, num_cpus=4, dashboard_port=8080)

# from tqdm import tqdm
# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")

2020-10-29 15:00:22,955	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8080


{'node_ip_address': '192.168.50.218',
 'raylet_ip_address': '192.168.50.218',
 'redis_address': '192.168.50.218:6379',
 'object_store_address': '/tmp/ray/session_2020-10-29_15-00-22_415111_9268/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-10-29_15-00-22_415111_9268/sockets/raylet',
 'webui_url': '127.0.0.1:8080',
 'session_dir': '/tmp/ray/session_2020-10-29_15-00-22_415111_9268',
 'metrics_export_port': 64369}

In [5]:
symbol='GLD'
# symbol = 'market'
# date = '2020-01-02'
# start_date = '2020-01-01'
start_date = '2020-09-01'
end_date = '2020-10-01'
# symbols=['SPY', 'QQQ']
tick_type='trades'

result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

thresh = {
    # GT thresholds
    'renko_reveral_multiple': 2,
    'duration_sec': 60*60,
    # LT thresholds
    'min_duration_sec': 15,
    'min_tick_count': 30,
}

horizon_mins=30

In [30]:
daily_bar_stats_df, labeled_bar_dates, stacked_bars_df = bm.bars_workflow_ray(
    result_path, 
    symbol, 
    start_date, 
    end_date, 
    thresh,
    imbalance_thresh=0.9
)

(pid=9302) GLD 2020-09-02 trying to get ticks from local file...
(pid=9304) GLD 2020-09-08 trying to get ticks from local file...
(pid=9303) GLD 2020-09-03 trying to get ticks from local file...
(pid=9305) GLD 2020-09-04 trying to get ticks from local file...
(pid=9303) GLD 2020-09-09 trying to get ticks from local file...
(pid=9302) GLD 2020-09-10 trying to get ticks from local file...
(pid=9305) GLD 2020-09-11 trying to get ticks from local file...
(pid=9304) GLD 2020-09-14 trying to get ticks from local file...
(pid=9303) GLD 2020-09-15 trying to get ticks from local file...
(pid=9305) GLD 2020-09-16 trying to get ticks from local file...
(pid=9304) GLD 2020-09-17 trying to get ticks from local file...
(pid=9302) GLD 2020-09-18 trying to get ticks from local file...
(pid=9303) GLD 2020-09-21 trying to get ticks from local file...
(pid=9302) GLD 2020-09-22 trying to get ticks from local file...
(pid=9304) GLD 2020-09-23 trying to get ticks from local file...
(pid=9305) GLD 2020-09-24

In [24]:
daily_bar_stats_df.bar_count.describe()

count     19.000000
mean      93.157895
std       38.163775
min       54.000000
25%       66.500000
50%       82.000000
75%       99.500000
max      183.000000
Name: bar_count, dtype: float64

In [26]:
stacked_bars_df.bar_trigger.value_counts()

volume_imbalance    893
renko_up            434
renko_down          427
gap_filler          222
duration             73
Name: bar_trigger, dtype: int64

In [29]:
daily_bar_stats_df.duration_min_mean.describe()

count    19.000000
mean     11.094340
std       3.368107
min       5.088525
25%       9.349128
50%      11.034496
75%      13.780951
max      16.352469
Name: duration_min_mean, dtype: float64

In [ ]:
bar_dates_df.price_wq50.diff().autocorr(1)

In [ ]:
# bar_dates_df.price_wmean.plot_bokeh(sizing_mode="scale_height")
bar_dates_df[200:400].price_wmean.plot()

In [ ]:
abs(bar_dates_df.label_rrr).value_counts()

In [ ]:
mh_dates_df = bar_dates_df.loc[bar_dates_df.close_at.dt.hour.between(13, 20)]

In [ ]:
mh_dates_df.label_outcome.value_counts()

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])